# Descripción del proyecto

Rusty Bargain es un servicio de venta de coches de segunda mano que está desarrollando una app para atraer a nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial, especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.

A Rusty Bargain le interesa:

* la calidad de la predicción
* la velocidad de la predicción
* el tiempo requerido para el entrenamiento

# Importar librerias

In [2]:
# Importar librerias
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
import lightgbm as lgb
from catboost import CatBoostRegressor


# Cargar base de datos

In [3]:
# Cargar datos
data = pd.read_csv('/datasets/car_data.csv')
data.head()


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


# Preprocesamiento

In [4]:
# Informacion del dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
# Descripcion del dataset
data.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
data.sample(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
345077,16/03/2016 14:48,11000,wagon,2012,manual,125,focus,60000,7,petrol,ford,no,16/03/2016 00:00,0,26133,04/04/2016 13:46
222320,29/03/2016 13:49,1200,sedan,1999,NaN,150,passat,150000,7,petrol,volkswagen,no,29/03/2016 00:00,0,55126,30/03/2016 18:33
129212,03/04/2016 16:39,950,sedan,1997,NaN,0,golf,125000,7,petrol,volkswagen,NaN,03/04/2016 00:00,0,56651,05/04/2016 15:46
348155,19/03/2016 20:37,14000,small,2013,manual,110,rio,50000,5,petrol,kia,no,19/03/2016 00:00,0,25358,26/03/2016 01:16
27623,10/03/2016 18:36,12900,suv,2010,manual,140,yeti,90000,9,gasoline,skoda,no,10/03/2016 00:00,0,38126,12/03/2016 09:16
240921,20/03/2016 10:39,4650,convertible,2002,manual,150,other,150000,11,petrol,saab,no,20/03/2016 00:00,0,53343,04/04/2016 05:49
257373,10/03/2016 12:47,200,small,1999,manual,60,punto,150000,6,petrol,fiat,yes,10/03/2016 00:00,0,21075,23/03/2016 08:16
54113,16/03/2016 23:38,7300,convertible,2002,manual,192,3er,150000,2,petrol,bmw,NaN,16/03/2016 00:00,0,34123,06/04/2016 00:17
109283,03/04/2016 18:25,1450,small,2003,manual,64,polo,70000,6,petrol,volkswagen,yes,03/04/2016 00:00,0,65549,06/04/2016 19:16
189375,11/03/2016 18:25,1,sedan,1990,manual,90,80,70000,6,petrol,audi,no,11/03/2016 00:00,0,55122,27/03/2016 19:15


In [7]:
# Manejo de Valores Faltantes
# Verificar valores faltantes en el dataset
missing_values = data.isnull().sum()
missing_values_percentage = (missing_values / data.shape[0]) * 100

missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_values_percentage
}).sort_values(by='Percentage (%)', ascending=False)

missing_data

# Imputar valores faltantes con la moda (categoría más frecuente)
for column in ['NotRepaired', 'VehicleType', 'FuelType', 'Gearbox', 'Model']:
    data[column].fillna(data[column].mode()[0], inplace=True)

# Verificar si ya no hay valores faltantes
missing_values_after = data.isnull().sum()
missing_values_after

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

In [8]:
# Transformación de Características y Eliminación de Columnas Irrelevantes
# Convertir las columnas DateCrawled, DateCreated y LastSeen en formato datetime
data['DateCrawled'] = pd.to_datetime(data['DateCrawled'], format='%d/%m/%Y %H:%M')
data['DateCreated'] = pd.to_datetime(data['DateCreated'], format='%d/%m/%Y %H:%M')
data['LastSeen'] = pd.to_datetime(data['LastSeen'], format='%d/%m/%Y %H:%M')

# Crear nuevas características basadas en estas fechas
data['DaysSinceCreated'] = (data['DateCrawled'].max() - data['DateCreated']).dt.days
data['DaysSinceLastSeen'] = (data['DateCrawled'].max() - data['LastSeen']).dt.days

# Eliminar columnas innecesarias
data = data.drop(columns=['NumberOfPictures', 'DateCrawled', 'DateCreated', 'LastSeen'])

# Verificar el dataset después de los cambios
data.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,PostalCode,DaysSinceCreated,DaysSinceLastSeen
0,480,sedan,1993,manual,0,golf,150000,0,petrol,volkswagen,no,70435,14,0
1,18300,coupe,2011,manual,190,golf,125000,5,gasoline,audi,yes,66954,14,0
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no,90480,24,2
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,91074,21,20
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,60437,7,1


In [9]:
# Identificación de Variables Categóricas y Codificación
# Código Utilizado para la Codificación de Variables Categóricas

# Identificar las variables categóricas
categorical_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

# Aplicar One-Hot Encoding a las variables categóricas
data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Verificar el dataset después de la codificación
data_encoded.head()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,PostalCode,DaysSinceCreated,DaysSinceLastSeen,VehicleType_convertible,VehicleType_coupe,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes
0,480,1993,0,150000,0,70435,14,0,False,False,...,False,False,False,False,False,False,False,True,False,False
1,18300,2011,190,125000,5,66954,14,0,False,True,...,False,False,False,False,False,False,False,False,False,True
2,9800,2004,163,125000,8,90480,24,2,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1500,2001,75,150000,6,91074,21,20,False,False,...,False,False,False,False,False,False,False,True,False,False
4,3600,2008,69,90000,7,60437,7,1,False,False,...,True,False,False,False,False,False,False,False,False,False


# Modelado y análisis de los modelos

In [10]:
# Separar la variable objetivo (Price) de las características
X = data_encoded.drop(columns=['Price'])
y = data_encoded['Price']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=12345)

# Verificar las dimensiones de los conjuntos de entrenamiento y prueba
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((248058, 310), (106311, 310), (248058,), (106311,))

In [11]:
# Crear una instancia del modelo de regresión lineal
model_lr = LinearRegression()

# Entrenar el modelo en el conjunto de entrenamiento
model_lr.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_lr = model_lr.predict(X_test)

# Calcular el RMSE (Root Mean Squared Error)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

print("RMSE para Regresión Lineal:", rmse_lr)

RMSE para Regresión Lineal: 3182.5681827729973


In [12]:

# Crear una instancia del modelo de bosque aleatorio
model_rf = RandomForestRegressor(n_estimators=100, random_state=12345)

# Entrenar el modelo en el conjunto de entrenamiento
model_rf.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_rf = model_rf.predict(X_test)

# Calcular el RMSE
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))

print("RMSE para Bosque Aleatorio:", rmse_rf)

RMSE para Bosque Aleatorio: 1713.0274114271135


In [13]:
# Crear una instancia del modelo LightGBM
model_lgb = lgb.LGBMRegressor(n_estimators=100, random_state=12345)

# Entrenar el modelo en el conjunto de entrenamiento
model_lgb.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_lgb = model_lgb.predict(X_test)

# Calcular el RMSE
rmse_lgb = np.sqrt(mean_squared_error(y_test, y_pred_lgb))

print("RMSE para LightGBM:", rmse_lgb)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012153 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1321
[LightGBM] [Info] Number of data points in the train set: 248058, number of used features: 292
[LightGBM] [Info] Start training from score 4414.500871
RMSE para LightGBM: 1822.3162838571827


In [14]:
# Crear una instancia del modelo CatBoost
model_cat = CatBoostRegressor(iterations=100, random_state=12345, verbose=0)

# Entrenar el modelo en el conjunto de entrenamiento
model_cat.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_cat = model_cat.predict(X_test)

# Calcular el RMSE
rmse_cat = np.sqrt(mean_squared_error(y_test, y_pred_cat))

print("RMSE para CatBoost:", rmse_cat)

RMSE para CatBoost: 1792.2614291823024


In [15]:
# Medir el tiempo de entrenamiento y predicción para Regresión Lineal
# Medir el tiempo de entrenamiento
start_time = time.time()
model_lr.fit(X_train, y_train)
train_time_lr = time.time() - start_time

# Medir el tiempo de predicción
start_time = time.time()
y_pred_lr = model_lr.predict(X_test)
predict_time_lr = time.time() - start_time

print("Tiempo de entrenamiento para Regresión Lineal:", train_time_lr)
print("Tiempo de predicción para Regresión Lineal:", predict_time_lr)

Tiempo de entrenamiento para Regresión Lineal: 14.400339841842651
Tiempo de predicción para Regresión Lineal: 0.277163028717041


In [16]:
# Medir el tiempo de entrenamiento y predicción para Bosque Aleatorio
# Medir el tiempo de entrenamiento
start_time = time.time()
model_rf.fit(X_train, y_train)
train_time_rf = time.time() - start_time

# Medir el tiempo de predicción
start_time = time.time()
y_pred_rf = model_rf.predict(X_test)
predict_time_rf = time.time() - start_time

print("Tiempo de entrenamiento para Bosque Aleatorio:", train_time_rf)
print("Tiempo de predicción para Bosque Aleatorio:", predict_time_rf)

Tiempo de entrenamiento para Bosque Aleatorio: 368.14562582969666
Tiempo de predicción para Bosque Aleatorio: 4.739103317260742


In [17]:
# Medir el tiempo de entrenamiento y predicción para LightGBM
# Medir el tiempo de entrenamiento
start_time = time.time()
model_lgb.fit(X_train, y_train)
train_time_lgb = time.time() - start_time

# Medir el tiempo de predicción
start_time = time.time()
y_pred_lgb = model_lgb.predict(X_test)
predict_time_lgb = time.time() - start_time

print("Tiempo de entrenamiento para LightGBM:", train_time_lgb)
print("Tiempo de predicción para LightGBM:", predict_time_lgb)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011197 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1321
[LightGBM] [Info] Number of data points in the train set: 248058, number of used features: 292
[LightGBM] [Info] Start training from score 4414.500871
Tiempo de entrenamiento para LightGBM: 2.2447309494018555
Tiempo de predicción para LightGBM: 0.4396548271179199


In [18]:
# Medir el tiempo de entrenamiento y predicción para CatBoost
# Medir el tiempo de entrenamiento
start_time = time.time()
model_cat.fit(X_train, y_train)
train_time_cat = time.time() - start_time

# Medir el tiempo de predicción
start_time = time.time()
y_pred_cat = model_cat.predict(X_test)
predict_time_cat = time.time() - start_time

print("Tiempo de entrenamiento para CatBoost:", train_time_cat)
print("Tiempo de predicción para CatBoost:", predict_time_cat)

Tiempo de entrenamiento para CatBoost: 1.1905407905578613
Tiempo de predicción para CatBoost: 0.03328704833984375


Antes de comparar los resultados de los modelos, voy a realizar un cross-validation y luego crearé nuevas versiones de los modelos originales con los hiperparametros modificados.

In [19]:
# Cross-validation para Bosque Aleatorio
# Crear una instancia del modelo
model_rf = RandomForestRegressor(n_estimators=100, random_state=12345)

# Aplicar la validación cruzada (3 folds)
cv_scores_rf = cross_val_score(model_rf, X_train, y_train, cv=3, scoring='neg_mean_squared_error')

# Convertir las puntuaciones a RMSE
rmse_cv_rf = np.sqrt(-cv_scores_rf)

# Mostrar los resultados
print("RMSE para Bosque Aleatorio (Cross-Validation):", rmse_cv_rf)
print("Media del RMSE para Bosque Aleatorio:", rmse_cv_rf.mean())

RMSE para Bosque Aleatorio (Cross-Validation): [1764.8758697  1766.93361565 1752.0565559 ]
Media del RMSE para Bosque Aleatorio: 1761.2886804162736


In [20]:
# Cross-validation para LightGBM
# Crear una instancia del modelo
model_lgb = lgb.LGBMRegressor(n_estimators=100, random_state=12345)

# Aplicar la validación cruzada (5 folds)
cv_scores_lgb = cross_val_score(model_lgb, X_train, y_train, cv=3, scoring='neg_mean_squared_error')

# Convertir las puntuaciones a RMSE
rmse_cv_lgb = np.sqrt(-cv_scores_lgb)

# Mostrar los resultados
print("RMSE para LightGBM (Cross-Validation):", rmse_cv_lgb)
print("Media del RMSE para LightGBM:", rmse_cv_lgb.mean())

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008803 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1305
[LightGBM] [Info] Number of data points in the train set: 165372, number of used features: 288
[LightGBM] [Info] Start training from score 4411.670984
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009047 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1302
[LightGBM] [Info] Number of data points in the train set: 165372, number of used features: 286
[LightGBM] [Info] Start training from score 4423.790908
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007579 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is

In [21]:
# Cross-validation para CatBoost
# Crear una instancia del modelo
model_cat = CatBoostRegressor(iterations=1000, learning_rate=0.03, depth=8, random_state=12345, verbose=0)

# Aplicar la validación cruzada (3 folds)
cv_scores_cat = cross_val_score(model_cat, X_train, y_train, cv=3, scoring='neg_mean_squared_error')

# Convertir las puntuaciones a RMSE
rmse_cv_cat = np.sqrt(-cv_scores_cat)

# Mostrar los resultados
print("RMSE para CatBoost (Cross-Validation):", rmse_cv_cat)
print("Media del RMSE para CatBoost:", rmse_cv_cat.mean())

RMSE para CatBoost (Cross-Validation): [1792.03067644 1782.11939541 1776.65804641]
Media del RMSE para CatBoost: 1783.602706085526


A continuación ajusté los hiperparametros para los modelos y ver como se modificaba su valor del RMSE, y también medir el tiempo tanto para la predicción como del entrenamiento.

In [22]:
# Nuevo modelo Random Forest con hiperparámetros optimizados
# Definir los hiperparámetros a buscar (reduciendo el número de opciones)
param_dist_rf = {
    'n_estimators': [100, 200],  # Reducido a 2 opciones
    'max_depth': [10, 20],       # Reducido a 2 opciones
    'min_samples_split': [2, 5], # Reducido a 2 opciones
    'min_samples_leaf': [1, 2],  # Reducido a 2 opciones
    'max_features': ['sqrt', 'log2']  # Reemplazado 'auto' por 1.0 y reducido a 2 opciones
}

# Crear la instancia del modelo con warm_start activado
model_rf = RandomForestRegressor(warm_start=True, random_state=12345)

# Configurar RandomizedSearchCV con 20 iteraciones y 3 folds para CV
random_search_rf = RandomizedSearchCV(estimator=model_rf, param_distributions=param_dist_rf, n_iter=20, cv=3, n_jobs=-1, verbose=2, random_state=42)

# Medir el tiempo de entrenamiento
start_time = time.time()
random_search_rf.fit(X_train, y_train)
train_time_rf = time.time() - start_time

# Mejor conjunto de hiperparámetros
print("Mejores hiperparámetros para Bosque Aleatorio:", random_search_rf.best_params_)

# Medir el tiempo de predicción
start_time = time.time()
y_pred_rf = random_search_rf.predict(X_test)
predict_time_rf = time.time() - start_time

# Calcular el RMSE
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))

# Imprimir resultados
print("RMSE para Bosque Aleatorio:", rmse_rf)
print("Tiempo de entrenamiento para Bosque Aleatorio:", train_time_rf, "segundos")
print("Tiempo de predicción para Bosque Aleatorio:", predict_time_rf, "segundos")

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END max_depth=10, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 1.2min
[CV] END max_depth=10, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 1.2min
[CV] END max_depth=10, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 1.2min
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time= 1.5min
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time= 1.5min
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time= 1.2min
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time= 2.8min
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=200

In [23]:
# Nuevo modelo LightGBM con hiperparámetros optimizados
# Definir los hiperparámetros a buscar
param_dist_lgb = {
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 500, 1000],
    'max_depth': [10, 15, 20],
    'feature_fraction': [0.7, 0.8, 0.9],
    'bagging_fraction': [0.7, 0.8, 0.9],
    'bagging_freq': [1, 5, 10]
}

# Crear la instancia del modelo
model_lgb = lgb.LGBMRegressor(random_state=12345, verbose=-1)  # Control de verbose en la instancia del modelo

# Configurar RandomizedSearchCV con 20 iteraciones y 3 folds para CV
random_search_lgb = RandomizedSearchCV(
    estimator=model_lgb,
    param_distributions=param_dist_lgb,
    n_iter=20,
    cv=3,
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# Configurar el callback para early stopping
early_stopping_callback = lgb.early_stopping(stopping_rounds=10)

# Medir el tiempo de entrenamiento
start_time = time.time()
random_search_lgb.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='rmse',
    callbacks=[early_stopping_callback]  # Pasar el callback aquí
)
train_time_lgb = time.time() - start_time

# Mejor conjunto de hiperparámetros
print("Mejores hiperparámetros para LightGBM:", random_search_lgb.best_params_)

# Medir el tiempo de predicción
start_time = time.time()
y_pred_lgb = random_search_lgb.predict(X_test)
predict_time_lgb = time.time() - start_time

# Calcular el RMSE
rmse_lgb = np.sqrt(mean_squared_error(y_test, y_pred_lgb))

# Imprimir resultados
print("RMSE para LightGBM:", rmse_lgb)
print("Tiempo de entrenamiento para LightGBM:", train_time_lgb, "segundos")
print("Tiempo de predicción para LightGBM:", predict_time_lgb, "segundos")

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[399]	valid_0's rmse: 1721.43	valid_0's l2: 2.96332e+06
[CV] END bagging_fraction=0.8, bagging_freq=10, feature_fraction=0.7, learning_rate=0.1, max_depth=20, n_estimators=1000, num_leaves=50; total time= 1.2min
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[662]	valid_0's rmse: 1705.41	valid_0's l2: 2.90842e+06
[CV] END bagging_fraction=0.8, bagging_freq=10, feat

In [24]:
# Nuevo modelo CatBoost con hiperparámetros optimizados
# Definir los hiperparámetros a buscar
param_dist_cat = {
    'depth': [6, 8, 10],
    'learning_rate': [0.01, 0.03, 0.1],
    'iterations': [500, 1000],
    'l2_leaf_reg': [3, 5, 7]
}

# Crear la instancia del modelo
model_cat = CatBoostRegressor(random_state=12345, verbose=0)

# Configurar RandomizedSearchCV con 20 iteraciones y 3 folds para CV
random_search_cat = RandomizedSearchCV(estimator=model_cat, param_distributions=param_dist_cat, n_iter=20, cv=3, n_jobs=-1, verbose=2, random_state=42)

# Medir el tiempo de entrenamiento
start_time = time.time()
random_search_cat.fit(X_train, y_train, early_stopping_rounds=10, eval_set=(X_test, y_test))
train_time_cat = time.time() - start_time

# Mejor conjunto de hiperparámetros
print("Mejores hiperparámetros para CatBoost:", random_search_cat.best_params_)

# Medir el tiempo de predicción
start_time = time.time()
y_pred_cat = random_search_cat.predict(X_test)
predict_time_cat = time.time() - start_time

# Calcular el RMSE
rmse_cat = np.sqrt(mean_squared_error(y_test, y_pred_cat))

# Imprimir resultados
print("RMSE para CatBoost:", rmse_cat)
print("Tiempo de entrenamiento para CatBoost:", train_time_cat, "segundos")
print("Tiempo de predicción para CatBoost:", predict_time_cat, "segundos")

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END depth=8, iterations=500, l2_leaf_reg=3, learning_rate=0.03; total time=  36.4s
[CV] END depth=8, iterations=500, l2_leaf_reg=3, learning_rate=0.03; total time=  37.7s
[CV] END depth=8, iterations=500, l2_leaf_reg=3, learning_rate=0.03; total time=  38.7s
[CV] END depth=6, iterations=1000, l2_leaf_reg=5, learning_rate=0.01; total time=  57.9s
[CV] END depth=6, iterations=1000, l2_leaf_reg=5, learning_rate=0.01; total time= 1.0min
[CV] END depth=10, iterations=1000, l2_leaf_reg=5, learning_rate=0.01; total time= 2.0min
[CV] END depth=10, iterations=1000, l2_leaf_reg=5, learning_rate=0.03; total time= 2.0min
[CV] END depth=10, iterations=1000, l2_leaf_reg=5, learning_rate=0.03; total time= 2.0min
[CV] END depth=10, iterations=1000, l2_leaf_reg=5, learning_rate=0.01; total time= 2.0min
[CV] END depth=10, iterations=1000, l2_leaf_reg=5, learning_rate=0.03; total time= 2.0min
[CV] END depth=6, iterations=500, l2_leaf_reg=5

# Conclusiones generales

Después de hacer una modelación con los hiperparametros predeterminados, una cross-validation y una modelación con los hiperparametros se obtuvieron resultados bastante interesantes.

Si creamos un modelo de Bosque aleatorio, LightGBM y CatBoost con sus hiperparametros predeterminados el mejor modelo es el bosque aleatorio respecto a su calidad de predicción(RMSE) con 1713.02  pero en relación a su tiempo para el entrenamiento(368.14 segundos) y velocidad de predición(4.73 segundos) no lo es, de hecho es el que tarda más tiempo, siendo el más rapido el CatBoost aunque también tiene una menor calidad de predicción(RMSE) de 1792.26. Entonces, como al cliente le interesa tanto la calidad de la predición como la velocidad, el modelo CatBoost sería el mejor aún cuando tu RMSE sea ligeramente inferior al modelo de Bosque aleatorio.

La segunda parte fue optimizar los hiperparametros para ver si se mejoraban las métricas y tomar una decisión final sobre el modelo a escoger. Practicamente los tiempos de entrenamiento se fueron por las nubes de los tres modelos, y la mejora en el RMSE fue muy realmente ligera, pero si al cliente lo que le importara sólo fuera la calidad de la predicción sin importarle el tiempo de entrenamiento y predicción, entonces el modelo LightGBM sería el ganador con RMSE de 1673.44.

Finalmente, se puede concluir que el modelo que puede ser usado para determinar el valor de mercado de un coche será el CatBoost sin optimizar los hiperparametros.